![bmw_logo](BMW-Logo-2020-present.jpg)

In [4]:
import pandas as pd
import numpy as np
import re
import os

import matplotlib.pyplot as plt
import matplotlib.ticker as mtick

import seaborn as sns

In [7]:
df = pd.read_csv(
    filepath_or_buffer="bmw.csv"
)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10781 entries, 0 to 10780
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   model         10781 non-null  object 
 1   year          10781 non-null  int64  
 2   price         10781 non-null  int64  
 3   transmission  10781 non-null  object 
 4   mileage       10781 non-null  int64  
 5   fuelType      10781 non-null  object 
 6   tax           10781 non-null  int64  
 7   mpg           10781 non-null  float64
 8   engineSize    10781 non-null  float64
dtypes: float64(2), int64(4), object(3)
memory usage: 758.2+ KB


#### 💾 Saving Plots

In [5]:
source = "source: https://www.kaggle.com/datasets/thedrzee/bmw-carsdataset\n"

with open("README.md", "w+") as readme:
    readme.write(source)

In [6]:
all_plots = os.listdir("figures")
print(all_plots)

with open("README.md", "a") as readme:    
    for plot in all_plots:
        if plot.endswith(".png"):
            description = (
                plot
                .replace(".png", "")
                .replace("_", " ")
                .capitalize()
            )
            readme.write(f"\n{description}\n")

            new_line = f"![{plot.replace('.png', '')}](figures/{plot})\n"
            readme.write(new_line)


[]
